In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display

In [2]:
scopus = list(MongoClient()["scopus_colombia"]["stage"].find({'DOI': {"$ne": np.nan}},{"_id":0,'Title':1,'Year':1, 'Source title':1,'Authors':1,"DOI":1}))
openalex = list(MongoClient()["colombia_udea"]["works"].find({},{"_id":0,"titles.title":1,"external_ids":1, "year_published":1, "authors.full_name":1, "source.names":1}))

In [3]:
#remove non doi ids
for i in range(len(openalex)):
    rec = openalex[i]
    for j in rec["external_ids"]:
        if j["source"] == "doi":
            openalex[i]["external_ids"] = j
            continue

In [4]:
openalex[0]

{'titles': [{'title': 'New records of Coronatella (Crustacea, Branchiopoda, Chydoridae) from Colombia with the first report of Coronatella undata and of the male of Coronatella monacantha'}],
 'external_ids': {'source': 'doi', 'id': '10.1590/1519-6984.254487'},
 'year_published': 2024,
 'authors': [{'full_name': 'Juan M. Fuentes-Reinés'},
  {'full_name': 'Pedro Eslava'},
  {'full_name': 'Lourdes M. A. Elmoor-Loureiro'},
  {'full_name': 'F. D. Rocha-S'},
  {'full_name': 'Deivis Suárez Rivero'}],
 'source': {'names': [{'lang': 'en', 'name': 'Brazilian Journal of Biology'}]}}

In [5]:
%%time
p = []
n = []
counter = 0
size=5000
p_dois = []
#positives
for i in scopus:
    
    for j in openalex:
        if i["DOI"] ==  j['external_ids']["id"]:
            p.append((i,j))
            p_dois.append(j['external_ids']["id"])
            counter+=1
            if counter%10 == 0:
                clear_output(wait=True)
                print(counter)
            continue
            
        if len(p) == size:
             break
    if len(p) == size:
         break

5000
CPU times: user 8min 53s, sys: 202 ms, total: 8min 53s
Wall time: 8min 53s


In [6]:
%%time
#negatives
n=[]
counter=0
n_dois = []
for i in scopus:
    for j in openalex:
        if i["DOI"] != j['external_ids']["id"]:
            if i["DOI"] not in p_dois and j['external_ids']["id"] not in p_dois:
                n.append((i,j))
                counter+=1
                if counter == size:
                    print(counter)
                    break
            if counter == size:
                break
            continue
    if counter == size:
        break


5000
CPU times: user 278 ms, sys: 1.15 ms, total: 279 ms
Wall time: 279 ms


In [7]:
len(p)

5000

In [8]:
len(n)

5000

In [9]:
p[0]

({'Authors': 'Jaramillo-Ayerbe F., Berrío-Muñoz J.',
  'Title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids',
  'Year': 1998,
  'Source title': 'Archives of Dermatology',
  'DOI': '10.1001/archderm.134.2.143'},
 {'titles': [{'title': 'Ivermectin for Crusted Norwegian Scabies Induced by Use of Topical Steroids'},
   {'title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids'}],
  'external_ids': {'source': 'doi', 'id': '10.1001/archderm.134.2.143'},
  'year_published': 1998,
  'authors': [{'full_name': 'Felipe Jaramillo-Ayerbe'},
   {'full_name': 'Joaquín Berrío-Muñoz'}],
  'source': {'names': [{'name': 'Archives of Dermatology', 'lang': 'en'}]}})

In [10]:
n[0]

({'Authors': 'Flórez G.A.D., Mombello E.E., Voss S.',
  'Title': 'Novel technique for the calculation of eddy current losses and Lorentz forces in foil winding transformers',
  'Year': 2017,
  'Source title': 'International Journal of Applied Electromagnetics and Mechanics',
  'DOI': '10..3233/JAE-160144'},
 {'titles': [{'title': 'New records of Coronatella (Crustacea, Branchiopoda, Chydoridae) from Colombia with the first report of Coronatella undata and of the male of Coronatella monacantha'}],
  'external_ids': {'source': 'doi', 'id': '10.1590/1519-6984.254487'},
  'year_published': 2024,
  'authors': [{'full_name': 'Juan M. Fuentes-Reinés'},
   {'full_name': 'Pedro Eslava'},
   {'full_name': 'Lourdes M. A. Elmoor-Loureiro'},
   {'full_name': 'F. D. Rocha-S'},
   {'full_name': 'Deivis Suárez Rivero'}],
  'source': {'names': [{'lang': 'en',
     'name': 'Brazilian Journal of Biology'}]}})

In [11]:
with open('dataset.pkl', 'wb') as f:
    pickle.dump({"p":p,"n":n}, f)